In [15]:
#import important libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
import math
import warnings
warnings.filterwarnings('ignore')

In [16]:
#read dataset csv file.
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
# find the features and observation in the data set.
data.shape

(10000, 14)

In [18]:
# find the datatypes of the features and target variable. 
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [19]:
#Dropping irrelevant data
dropped = ["RowNumber", "CustomerId", "Surname"]
for i in range(len(dropped)):
    del data[dropped[i]]
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [20]:
#Handling categorical data
categorical_data = []
for name in list(data.columns):
    if is_numeric_dtype(data[name]):
        pass
    else:
        categorical_data.append(name)


data = pd.get_dummies(data, columns=categorical_data)
data.head()


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [21]:
#Getting data shape
data.shape

(10000, 14)

In [22]:
#Dividing data
y = pd.DataFrame(data.Exited) 
y.shape

(10000, 1)

In [23]:
#Dividing data
x = data.drop(columns = ["Exited"])
x.shape

(10000, 13)

In [25]:
class MultiNeuralNetwork:

    def __init__(self):
        # Learning Rate
        self.l_rate = 0.001
        # Total iterations
        self.echops = 2000

        #Training our data by stochastic gradient descent
    def train(self, x_train_data, y_train_data):

        #setting layer and number of neurons in those layers
        layers = [x_train_data.shape[1], 50, 44, 65, 43, 1]
        
        #Initializing different values
        weight = []
        baised = []
        a = [0] * len(layers)
        z = [0] * len(layers)
        dg = [0] * len(layers)
        da = [0] * len(layers)
        dz = [0] * len(layers)
        db = [0] * len(layers)
        dw = [0] * len(layers)
        
        a[0] = x_train_data.T 
        m = x_train_data.shape[0]
        n = x_train_data.shape[1]

        for j in range(1, len(layers)):
            
            weight.append(np.random.rand(layers[j], layers[j-1]) * 0.001 ) 
            baised.append(np.zeros(((layers[j], 1)) ))
 
        for k in range(self.echops):
            for l in range(len(y_train_data)): 
                for i in range(len(layers) - 1):  

                    z[i] = np.dot(weight[i], a[i]) + baised[i] 
                    a[i+1] = 1 / (1 + np.exp(-z[i])) 

                    # d_g = a * (1 - a)
                    dg[i] = (1 / (1 + np.exp(-z[i]))) * (1 - (1 / (1 + np.exp(-z[i])))) 

                      # Loss Funtion
                    # da = -(y/a) + ((1-y)/(1-a))
                    da[i] = (-(y_train_data.T / a[i+1]) + ((1 - y_train_data.T) / (1 - a[i+1]))) 

                    # dz = da * d_g
                    dz[i] = (da[i] * dg[i]) # 5x209

                    #dw = (dz * a) / m
                    dw[i] = np.dot(dz[i], a[i].T)

                    # db = np.sum(dz)
                    db[i] = np.sum(dz[i], axis=1, keepdims=True) / m 

                    # w = w - alpha * dw
                    # b = b - alpha * db
                    weight[i] =  weight[i] - np.dot(self.l_rate, dw[i]) 
                    baised[i] = baised[i] - np.dot(self.l_rate, db[i]) 

        return [weight, baised]
    
    #Defining fuction to get predicted data
    def predict(self, x_test_data, parameters):

        a = [0] * 6
        a[0] = x_test_data.T
        z = [0] * 6
        a = [0] * 6
        for i in range(4):  
            z[i] = np.dot(parameters[0][i], a[i]) + parameters[1][i]
            a[i+1] = 1 / (1 + np.exp(-z[i])) 
        return a[-1]
        return y_predict
    
    #Defining function to calculate accuracy 
    def accuracy(self, y_data_test, y_pred_test):

        y_pred_test = np.nan_to_num(y_pred_test)
    
        test_accuracy = 100 - (np.mean(np.abs(y_pred_test - y_data_test)) * 100)        
        return test_accuracy


def main():

    #creating object for class
    mnn = MultiNeuralNetwork()
    
    #setting data size for test and train data
    train_size = int(np.ceil((len(y) * 0.80)))
    
    #Training data
    x_train_data = np.array(x[:train_size])
    y_train_data = np.array(y[:train_size])
    
    #Testing data
    x_test_data = np.array(x[train_size:])
    y_test_data = np.array(y[train_size:])
    
    
    parameters = mnn.train(x_train_data, y_train_data)
    
    y_prediction = mnn.predict(x_test_data, parameters)
    
    y_prediction_train = mnn.predict(x_train_data, parameters)
    
    accuracy = mnn.accuracy(y_test_data, y_prediction)
    accuracy_train = mnn.accuracy(y_train_data, y_prediction_train)

    print("Train accuracy: ", accuracy_train)
    print("Test accuracy: ", accuracy)


if __name__ == '__main__':
    main()

Train accuracy:  79.4125
Test accuracy:  80.5
